In [174]:
import numpy as np
np.random.seed(18282783)
# get all possible directions
min_dir, max_dir = -1, 1
xs, ys = np.meshgrid(np.arange(min_dir,max_dir+1), np.arange(min_dir,max_dir+1))
d_coord = np.transpose(np.array([xs.ravel(), ys.ravel()]))
d_coord = d_coord[(d_coord[:,0] != 0) | (d_coord[:,1] != 0)]

# print array with commas
# print(repr(t))

In [181]:
def random_walk(path_length = 100, board_size = 30, trials = 100):
    def on_board(x, y):
        return 0 <= x < board_size and 0 <= y < board_size

    path = np.zeros((path_length,2))
    path[0] = [board_size//2, board_size//2]

    def in_path(x, y):
        return any(np.equal(path, [x, y]).all(1))

    for i in range(path_length-1):
        x_nxt, y_nxt = path[i]
        found = False
        # print(i)
        for j in range(trials):
            x_nxt, y_nxt = path[i] + d_coord[np.random.randint(d_coord.shape[0])]
            # print(x_nxt, y_nxt)
            # print([x_nxt, y_nxt] in path, path[:i+1])
            if on_board(x_nxt, y_nxt) and not in_path(x_nxt, y_nxt):
                path[i+1] = [x_nxt, y_nxt]
                found = True
                break
        if not found:
            break
    return path


print(random_walk())

[[15. 15.]
 [14. 16.]
 [14. 17.]
 [14. 18.]
 [15. 17.]
 [16. 16.]
 [17. 15.]
 [16. 14.]
 [17. 14.]
 [16. 15.]
 [15. 16.]
 [16. 17.]
 [15. 18.]
 [15. 19.]
 [14. 19.]
 [13. 18.]
 [12. 17.]
 [12. 18.]
 [11. 18.]
 [12. 19.]
 [11. 19.]
 [10. 20.]
 [ 9. 20.]
 [10. 19.]
 [10. 18.]
 [ 9. 18.]
 [ 8. 18.]
 [ 7. 17.]
 [ 6. 17.]
 [ 5. 17.]
 [ 5. 18.]
 [ 6. 18.]
 [ 6. 19.]
 [ 5. 20.]
 [ 6. 21.]
 [ 7. 22.]
 [ 6. 23.]
 [ 5. 24.]
 [ 5. 25.]
 [ 6. 25.]
 [ 7. 25.]
 [ 6. 26.]
 [ 5. 26.]
 [ 4. 25.]
 [ 4. 24.]
 [ 3. 25.]
 [ 3. 24.]
 [ 4. 23.]
 [ 3. 23.]
 [ 4. 22.]
 [ 5. 22.]
 [ 5. 23.]
 [ 6. 24.]
 [ 7. 24.]
 [ 8. 25.]
 [ 8. 24.]
 [ 9. 24.]
 [10. 23.]
 [11. 22.]
 [10. 21.]
 [ 9. 22.]
 [ 9. 21.]
 [10. 22.]
 [11. 21.]
 [11. 20.]
 [12. 21.]
 [12. 20.]
 [13. 20.]
 [14. 21.]
 [14. 22.]
 [15. 23.]
 [14. 23.]
 [13. 22.]
 [13. 23.]
 [12. 24.]
 [13. 25.]
 [12. 25.]
 [13. 26.]
 [12. 26.]
 [11. 26.]
 [10. 25.]
 [10. 26.]
 [10. 27.]
 [11. 28.]
 [12. 27.]
 [12. 28.]
 [13. 28.]
 [13. 29.]
 [12. 29.]
 [11. 29.]
 [10. 28.]

In [26]:
import numpy as np

max_visibility_distance = 7
default_visibility_distance = 4
board_size = 6

chebyshev_list = [[] for i in range(max_visibility_distance+1)]

for d in range(1, max_visibility_distance+1):
    xs, ys = np.meshgrid([d], np.arange(-d, d+1))
    chebyshev_list[d] = np.array([xs.ravel(), ys.ravel()])

# print(np.array(chebyshev_list))

# directions 0>,1^,2<,3v

# for each direction, contains multipliers for xs, ys to watch in clockwise direction around map
# x, y

# counter-clockwise codes for indices
cc = np.array([[[1, 0], [1, 1]],
               [[0, 1], [1, 1]],
               [[1, 0], [1, -1]],
               [[0, 1], [-1, 1]]])
len_cc = len(cc)

def get_chebyshev_indices(direction, distance):
    assert 0 <= direction <= 3
    assert 0 <= distance <= max_visibility_distance
    if distance == 0:
        return np.empty(0,int)
    indices = np.empty((0,2), int)
    for i in range(direction-1, (direction+1)+1):
        ind_cc = (len_cc + i) % len_cc
        perm, mult = cc[ind_cc]
        chebyshev = chebyshev_list[distance]
        perm = mult[:, np.newaxis] * np.array([chebyshev[i] for i in perm])
        indices = np.append(indices, perm.T, axis=0)
    indices = indices.astype(int)
    index = np.unique(indices.astype(int), axis=0, return_index=True)[1]
    return np.array([indices])

chebyshev_indices = [[get_chebyshev_indices(distance=distance, direction=direction)
                      for distance in range(0, max_visibility_distance)]
                     for direction in range(0, 4)]

lst = chebyshev_indices[0][1]
# print(lst)
# lst_c = np.zeros(lst.shape)
# for i in range(len(lst)-1):
#     print(lst[i+1]-lst[i])
#
# print(lst_c)

def get_visible_indices(x, y, distance, direction):
    ind = chebyshev_indices[direction][distance] + np.array([x,y])
    avail_ind = ind[(0 <= ind[:,0]) & (ind[:,0] < board_size) & (0 <= ind[:,1]) & (ind[:,1] < board_size)]
    return avail_ind

field = np.array([[set() for i in range(board_size)] for j in range(board_size)])

np.random.seed(192)
id = 2
xs, ys = np.random.random_integers(0, board_size-1, (2, 10))
for i, j in zip(xs, ys):
    field[i][j].add(id)

# print(field)

def obstacle_at_dist(x, y, direction, id):
    for distance in range(1, max_visibility_distance+1):
        avail_ind = get_visible_indices(x=x, y=y, distance=distance, direction=direction)
        if len(chebyshev_indices[direction][distance]) != avail_ind.shape[0]:
            return distance
        for i, j in avail_ind:
            if id in field[i][j]:
                return distance
    return default_visibility_distance

snakes_number = 20
directions = 7
# 0-cells from directions store snake's scores+

# snakes_eyes = np.random.rand((snakes_number, directions, max_visibility_distance+1))

# get_visible_indices(x=2, y=4, distance=2, direction=2)
obstacle_at_dist(x=3, y=3, direction=0, id = 2)

[[-1 -1]
 [-1  0]
 [-1  1]
 [ 0  1]
 [ 1 -1]
 [ 1  0]
 [ 1  1]]


<ipython-input-26-b0c6010d1040>:62: DeprecationWarning: This function is deprecated. Please call randint(0, 5 + 1) instead
  xs, ys = np.random.random_integers(0, board_size-1, (2, 10))


(2, 2)

In [33]:
import numpy as np
arr = np.empty((0,3), int)
print("Empty array:")
print(arr)
arr = np.append(arr, np.array([[10,20,30],[1,2,3]]), axis=0)
# arr = np.append(arr, np.array([[40,50,60]]), axis=0)
print("After adding two new arrays:")
print(arr)

Empty array:
[]
After adding two new arrays:
[[10 20 30]
 [ 1  2  3]]


In [36]:
# pylint: disable=no-member
""" scatter using MarkersVisual """

import numpy as np
import sys

from vispy import app, visuals, scene


# build your visuals, that's all
Scatter3D = scene.visuals.create_visual_node(visuals.MarkersVisual)

# The real-things : plot using scene
# build canvas
canvas = scene.SceneCanvas(keys='interactive', show=True)

# Add a ViewBox to let the user zoom/rotate
view = canvas.central_widget.add_view()
view.camera = 'turntable'
view.camera.fov = 45
view.camera.distance = 500

# data
n = 500
pos = np.zeros((n, 3))
colors = np.ones((n, 4), dtype=np.float32)
radius, theta, dtheta = 1.0, 0.0, 10.5 / 180.0 * np.pi
for i in range(500):
    theta += dtheta
    x = 0.0 + radius * np.cos(theta)
    y = 0.0 + radius * np.sin(theta)
    z = 1.0 * radius
    r = 10.1 - i * 0.02
    radius -= 0.45
    pos[i] = x, y, z
    colors[i] = (i/500, 1.0-i/500, 0, 0.8)

# plot ! note the parent parameter
p1 = Scatter3D(parent=view.scene)
p1.set_gl_state('translucent', blend=True, depth_test=True)
p1.set_data(pos, face_color=colors, symbol='o', size=10,
            edge_width=0.5, edge_color='blue')

# run
# if __name__ == '__main__':
if sys.flags.interactive != 1:
    app.run()

ModuleNotFoundError: No module named 'vispy'

In [6]:
import numpy as np
a = np.array([[1,2,3],[1,3,4],[5,6,7]])
def f(p): return p[0] * p[1]
print(np.apply_along_axis(f, 1, a))

[ 2  3 30]


In [3]:
from PyQt5.QtWidgets import QApplication
from pyqtgraph.Qt import QtCore
import pyqtgraph.opengl as gl
import sys
import numpy as np

x = np.array([1,2])
y = np.array([1,2])
z = np.array([[1,2],[3,4]])
colors = np.array([[[0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5]],
                   [[0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5]]]).astype(float)

app = QApplication([])
w = gl.GLViewWidget()
w.show()
board_size = 20
ax = gl.GLAxisItem()
ax.setSize(x=board_size, y=board_size, z=20)
w.addItem(ax)

sp3 = gl.GLSurfacePlotItem(x=x, y=y, z=z, color=colors)
w.addItem(sp3)

if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
    QApplication.instance().exec_()

[19:50:16]  

    |==============================>>
    |  Traceback (most recent call last):
    |    File "/home/danila/miniconda3/envs/myenv/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    |      "__main__", mod_spec)
    |    File "/home/danila/miniconda3/envs/myenv/lib/python3.7/runpy.py", line 85, in _run_code
    |      exec(code, run_globals)
    |    File "/home/danila/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    |      app.launch_new_instance()
    |    File "/home/danila/miniconda3/envs/myenv/lib/python3.7/site-packages/traitlets/config/application.py", line 845, in launch_instance
    |      app.start()
    |    File "/home/danila/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 612, in start
    |      self.io_loop.start()
    |    File "/home/danila/miniconda3/envs/myenv/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
    |      self.asyncio_loop